Tracking Objects in Video with Particle Filters
===============================================

Import libraries

In [81]:
import numpy as np
import cv2

# Repeatability
np.random.seed(0)

VFILENAME = "walking.mp4"
HEIGHT = 406
WIDTH = 722

Load video frames from file

In [82]:
def get_frames(filename):
    video = cv2.VideoCapture(filename)
    while video.isOpened():
        ret,frame = video.read()
        if ret:
            yield frame
        else:
            break
    video.release()
    yield None

Creating a particle cloud

In [83]:
NUM_PARTICLES = 150
VEL_RANGE = 0.5
def initialize_particles():
    particles = np.random.rand(NUM_PARTICLES, 4)
    particles = particles*np.array( (WIDTH, HEIGHT, VEL_RANGE, VEL_RANGE))
    #velocities to be centered at zero
    particles[:,2:4] -= VEL_RANGE/2.0 
    print(particles[:20,:])
    return particles

Moving particles according to their velocity state

In [84]:
def apply_velocity(particles):
    particles[:,0] += particles[:,2]
    particles[:,1] += particles[:,3]
    return particles

Prevent particles from falling off the edge of the video frame

In [85]:
def enforce_edges(particles):
    for i in range(NUM_PARTICLES):
        #make sure x is not greater than width-1
        #make sure x is not less than 0
        particles[i,0] = max(0, min(WIDTH-1, particles[i,0]))
        #make sure y is not greater than height-1
        #make sure y is not less than 0
        particles[i,1] = max(0, min(HEIGHT-1, particles[i,1]))
        
    return particles

Measure each particle's quality

In [86]:
TARGET_COLOR = np.array([156, 74, 38])
def compute_errors(particles, frame):
    #store color differences
    errors = np.zeros(NUM_PARTICLES)
    for i in range(NUM_PARTICLES):
        x = int(particles[i,0])
        y = int(particles[i,1])
        pixel_color = frame[y, x, :]
        #mean squared diff
        errors[i] = np.sum((TARGET_COLOR - pixel_color)**2)
    return errors

Assign weights to the particles based on their quality of match

In [1]:
def compute_weights(errors):
    #invert errors to get weights
    weights = np.max(errors) - errors
    #prevent particles from staying at edge
    weights[(particles[:,0] == 0) | 
            (particles[:,0] == WIDTH-1) |
            (particles[:,1] == 0) |
            (particles[:,1] == HEIGHT-1)] = 0.0
    #raise the power of the weights to make them more sensitive
    #large weights get larger and gets resampled more
    weights = weights**4
    return weights

Resample particles according to their weights

In [88]:
def resample(particles, weights):
    probabilities = weights/ np.sum(weights)
    index_numbers = np.random.choice(NUM_PARTICLES, size=NUM_PARTICLES, p=probabilities)
    particles = particles[index_numbers, :]
    
    #single best guess
    x = np.mean(particles[:,0])
    y = np.mean(particles[:,1])
    return particles, (int(x), int(y))

Fuzz the particles

In [89]:
POS_SIGMA = 1.0
VEL_SIGMA = 0.5 #std
#why add noise
#when all the particles were on the same pixel 
#they had the same error which caused the weights to be zero
#then we divide by sum of weights which is zero so this caused the program to crash
def apply_noise(particles):
    noise = np.concatenate((                  
                            (np.random.normal(0.0, POS_SIGMA, (NUM_PARTICLES,1)),
                            (np.random.normal(0.0, POS_SIGMA, (NUM_PARTICLES,1)),
                            (np.random.normal(0.0, VEL_SIGMA, (NUM_PARTICLES,1)),
                            (np.random.normal(0.0, VEL_SIGMA, (NUM_PARTICLES,1))
                            ), axis = 1)
    particles += noise
    return particles

Display the video frames

In [90]:
def display(frame, particles, location):
    if len(particles)>0:
        for i in range(NUM_PARTICLES):
            x = int(particles[i,0])
            y = int(particles[i,1])
            cv2.circle(frame, (x,y), 1, (0,255,0), 1)
        if len(location) >0:
            cv2.circle(frame, location, 15, (0,0,255), 5)
        cv2.imshow('frame', frame)
        if cv2.waitKey(30) == 27: #esc key:
            if cv2.waitKey(0) == 27:
                return True
    return False

Main routine

In [91]:
particles = initialize_particles()

for frame in get_frames(VFILENAME):
    if frame is None: break

    particles = apply_velocity(particles)
    particles = enforce_edges(particles)
    errors = compute_errors(particles, frame)
    weights = compute_weights(errors)
    particles, location = resample(particles, weights)
    particles = apply_noise(particles)
    terminate = display(frame, particles, location)
    if terminate:
        break
cv2.destroyAllWindows()


[[ 3.96243350e+02  2.90366883e+02  5.13816880e-02  2.24415915e-02]
 [ 3.05878765e+02  2.62233010e+02 -3.12063944e-02  1.95886500e-01]
 [ 6.95764513e+02  1.55677257e+02  1.45862519e-01  1.44474599e-02]
 [ 4.10128173e+02  3.75792235e+02 -2.14481971e-01 -2.06435350e-01]
 [ 1.45976830e+01  3.38043657e+02  1.39078375e-01  1.85006074e-01]
 [ 7.06562443e+02  3.24458377e+02 -1.92603189e-02  1.40264588e-01]
 [ 8.53941355e+01  2.59807935e+02 -1.78323356e-01  2.22334459e-01]
 [ 3.76774488e+02  1.68352748e+02 -1.17722194e-01  1.37116845e-01]
 [ 3.29340540e+02  2.30784183e+02 -2.40605100e-01  5.88177485e-02]
 [ 4.41933112e+02  2.50475203e+02  2.21874039e-01  9.09101496e-02]
 [ 2.59564704e+02  1.77434973e+02  9.88155980e-02 -2.19887264e-01]
 [ 4.81405569e+02  2.72278975e+02 -1.44808719e-01 -1.85536851e-01]
 [ 2.27739269e+02  1.47666573e+02  3.50983852e-02 -3.06992433e-02]
 [ 7.13605911e+02  4.14301932e+01 -1.45561622e-01 -1.69345241e-01]
 [ 4.71544211e+02  1.02836391e+02 -1.68446136e-02 -1.27787204e

/home/rhyme/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


ValueError: probabilities contain NaN